In [1]:
# Setup: imports and data load
import json, random
from typing import List, Tuple, Dict
from collections import Counter, defaultdict
from math import log

SEED = 42
random.seed(SEED)

# Load tokenized dataset produced in lab1
with open('../lab1/gujarati_corpus_tokenized.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract sentences as token lists (keep punctuation as tokens)
all_sentences_tokens: List[List[str]] = []
for doc in data:
    for s in doc.get('sentences', []):
        tokens = s.get('words', [])
        if tokens:
            all_sentences_tokens.append(tokens)

print(f"Total sentences available: {len(all_sentences_tokens)}")

Total sentences available: 1631


In [2]:
# Create 100/100/remaining splits (validation/test/training)
num_sentences = len(all_sentences_tokens)
indices = list(range(num_sentences))
random.shuffle(indices)

val_size = min(100, num_sentences)
test_size = min(100, max(0, num_sentences - val_size))

val_idx = set(indices[:val_size])
test_idx = set(indices[val_size:val_size + test_size])

val_set = [all_sentences_tokens[i] for i in val_idx]
test_set = [all_sentences_tokens[i] for i in test_idx]
train_set = [all_sentences_tokens[i] for i in indices if i not in val_idx and i not in test_idx]

print(f"Split sizes -> train: {len(train_set)}, val: {len(val_set)}, test: {len(test_set)}")

Split sizes -> train: 1431, val: 100, test: 100


In [3]:
# N-gram counting helpers (BOS/EOS markers)
from typing import Iterable

def add_markers(sent: List[str], n: int) -> List[str]:
    return ['<s>'] * (n-1) + sent + ['</s>']

def extract_ngrams(sent: List[str], n: int) -> Iterable[Tuple[str, ...]]:
    tokens = add_markers(sent, n)
    for i in range(len(tokens) - n + 1):
        yield tuple(tokens[i:i+n])

def build_ngram_counts(sentences: List[List[str]], n: int):
    ngram_counts = Counter()
    context_counts = Counter()
    vocab = set()
    for sent in sentences:
        vocab.update(sent)
        for ng in extract_ngrams(sent, n):
            ngram_counts[ng] += 1
            if n > 1:
                context_counts[ng[:-1]] += 1
    # include markers in vocab
    vocab.update({'<s>', '</s>'})
    return ngram_counts, context_counts, vocab

In [4]:
# Good-Turing smoothing utilities

def compute_Nc(ngram_counts: Counter) -> Dict[int, int]:
    Nc = Counter()
    for _, c in ngram_counts.items():
        Nc[c] += 1
    return Nc

class GoodTuringModel:
    def __init__(self, n: int, ngram_counts: Counter, context_counts: Counter, vocab: set):
        self.n = n
        self.ngram_counts = ngram_counts
        self.context_counts = context_counts
        self.vocab = vocab
        self.V = len(vocab)
        self.N = sum(ngram_counts.values())  # total seen n-gram tokens
        self.unique_seen = len(ngram_counts) # number of distinct seen n-grams
        self.Nc = compute_Nc(ngram_counts)
        self.N1 = self.Nc.get(1, 0)
        # Space of all possible n-grams
        if n == 1:
            self.total_possible = self.V
            self.unseen_possible = self.V - self.unique_seen
        else:
            self.total_possible = self.V ** n
            self.unseen_possible = max(0, self.total_possible - self.unique_seen)
        # Cumulative mass for unseen
        self.mass_unseen = (self.N1 / self.N) if self.N > 0 else 0.0
        # Per unseen n-gram probability
        self.P_unseen = (self.mass_unseen / self.unseen_possible) if self.unseen_possible > 0 else 0.0

    def adjusted_count(self, c: int) -> float:
        # Good-Turing adjusted count c* = (c+1) * N_{c+1} / N_c
        if c == 0:
            return self.mass_unseen  # distribute later as probability
        Nc = self.Nc.get(c, 0)
        Ncp1 = self.Nc.get(c+1, 0)
        if Nc == 0:
            return c  # fallback to MLE if no neighbor counts
        return (c + 1) * (Ncp1 / Nc)

    def ngram_probability(self, ngram: Tuple[str, ...]) -> float:
        c = self.ngram_counts.get(ngram, 0)
        if c == 0:
            return self.P_unseen
        c_star = self.adjusted_count(c)
        # For conditional models (n>=2), normalize by context count; for unigram by N
        if self.n == 1:
            return c_star / self.N
        else:
            context = ngram[:-1]
            context_count = self.context_counts.get(context, 0)
            # If context never seen, back off to unigram-like unseen per space
            if context_count == 0:
                return self.P_unseen
            # Scale c* to probability roughly as MLE replacement
            return c_star / context_count

    def sentence_logprob(self, sentence: List[str]) -> float:
        tokens = ['<s>'] * (self.n - 1) + sentence + ['</s>']
        lp = 0.0
        for i in range(len(tokens) - self.n + 1):
            ng = tuple(tokens[i:i+self.n])
            p = self.ngram_probability(ng)
            if p <= 0:
                return float('-inf')
            lp += log(p)
        return lp

In [5]:
# Build counts for n=1..4 on training set and init GT models
orders = [1,2,3,4]
counts = {}
models_gt = {}
for n in orders:
    ng_counts, ctx_counts, vocab = build_ngram_counts(train_set, n)
    counts[n] = (ng_counts, ctx_counts, vocab)
    models_gt[n] = GoodTuringModel(n, ng_counts, ctx_counts, vocab)
    print(f"n={n}: seen n-grams={len(ng_counts)}, tokens={sum(ng_counts.values())}, V={len(vocab)}; N1={models_gt[n].N1}, unseen_possible≈{models_gt[n].unseen_possible}")

# Helper to compute corpus perplexity
def corpus_perplexity(model: GoodTuringModel, corpus: List[List[str]]) -> float:
    total_logprob = 0.0
    total_tokens = 0
    for sent in corpus:
        lp = model.sentence_logprob(sent)
        if lp == float('-inf'):
            return float('inf')
        total_logprob += lp
        total_tokens += len(sent) + 1  # include </s>
    if total_tokens == 0:
        return float('inf')
    avg_logprob = total_logprob / total_tokens
    # natural log perplexity
    return pow(2.718281828, -avg_logprob)

# Evaluate on validation and test
for split_name, split in [('val', val_set), ('test', test_set)]:
    print(f"\nPerplexities on {split_name} set:")
    for n in orders:
        ppl = corpus_perplexity(models_gt[n], split)
        print(f"  {n}-gram GT perplexity: {ppl:.3f}")

n=1: seen n-grams=7700, tokens=21944, V=7701; N1=5339, unseen_possible≈1
n=2: seen n-grams=17692, tokens=21944, V=7701; N1=16254, unseen_possible≈59287709
n=3: seen n-grams=20282, tokens=21944, V=7701; N1=19573, unseen_possible≈456710872819
n=4: seen n-grams=21026, tokens=21944, V=7701; N1=20625, unseen_possible≈3517130587749775

Perplexities on val set:
  1-gram GT perplexity: inf
  2-gram GT perplexity: inf
  3-gram GT perplexity: inf
  4-gram GT perplexity: inf

Perplexities on test set:
  1-gram GT perplexity: inf
  2-gram GT perplexity: inf
  3-gram GT perplexity: inf
  4-gram GT perplexity: inf


In [6]:
# Top-100 frequency tables: C, Nc, C*, P_MLE
import pandas as pd

freq_tables = {}
for n in orders:
    ng_counts, ctx_counts, _ = counts[n]
    gt = models_gt[n]
    rows = []
    for ng, c in ng_counts.most_common(100):
        Nc = gt.Nc.get(c, 0)
        c_star = gt.adjusted_count(c)
        if n == 1:
            p_mle = c / gt.N if gt.N > 0 else 0
        else:
            ctx = ng[:-1]
            denom = ctx_counts.get(ctx, 0)
            p_mle = c / denom if denom > 0 else 0
        rows.append({
            'ngram': ' '.join(ng),
            'C': c,
            'Nc': Nc,
            'C*': c_star,
            'P_MLE': p_mle
        })
    df = pd.DataFrame(rows)
    freq_tables[n] = df
    print(f"\nTop 10 for {n}-gram:")
    display(df.head(10))


Top 10 for 1-gram:


,ngram,C,Nc,C*,P_MLE
0,</s>,1431,1,0.0,0.065211
1,છે,808,1,0.0,0.036821
2,",",762,1,0.0,0.034725
3,અને,387,1,0.0,0.017636
4,આ,241,1,0.0,0.010983
5,કે,193,1,0.0,0.008795
6,પણ,189,1,0.0,0.008613
7,માટે,177,1,0.0,0.008066
8,-,150,1,0.0,0.006836
9,કરી,126,1,0.0,0.005742



Top 10 for 2-gram:


,ngram,C,Nc,C*,P_MLE
0,છે </s>,521,1,0.0,0.644802
1,<s> આ,107,1,0.0,0.074773
2,"છે ,",99,1,0.0,0.122525
3,હતી </s>,71,1,0.0,0.717172
4,હતો </s>,57,1,0.0,0.740260
5,છે કે,51,1,0.0,0.063119
6,"કે ,",45,1,0.0,0.233161
7,હતા </s>,42,1,0.0,0.724138
8,કરે છે,38,1,0.0,0.883721
9,છે અને,37,1,38.0,0.045792



Top 10 for 3-gram:


,ngram,C,Nc,C*,P_MLE
0,<s> <s> આ,107,1,0.000000,0.074773
1,શકે છે </s>,28,1,0.000000,0.777778
2,કરે છે </s>,24,1,0.000000,0.631579
3,આવી છે </s>,21,1,0.000000,0.840000
4,<s> <s> જો,20,1,21.000000,0.013976
5,આવે છે </s>,19,1,20.000000,0.575758
6,<s> <s> એક,16,1,0.000000,0.011181
7,<s> <s> પરંતુ,15,3,5.333333,0.010482
8,થાય છે </s>,15,3,5.333333,0.789474
9,રહી છે </s>,15,3,5.333333,0.937500



Top 10 for 4-gram:


,ngram,C,Nc,C*,P_MLE
0,<s> <s> <s> આ,107,1,0.0,0.074773
1,<s> <s> <s> જો,20,1,0.0,0.013976
2,<s> <s> <s> એક,16,1,0.0,0.011181
3,<s> <s> <s> પરંતુ,15,1,16.0,0.010482
4,<s> <s> <s> તે,14,1,15.0,0.009783
5,<s> <s> <s> જોકે,13,1,14.0,0.009085
6,કરી શકે છે </s>,12,2,6.5,0.857143
7,કરવામાં આવી છે </s>,12,2,6.5,0.923077
8,<s> <s> <s> ત્યારે,11,2,12.0,0.007687
9,<s> <s> <s> જ્યારે,11,2,12.0,0.007687


In [7]:
# Deleted interpolation for 4-gram (Jelinek-Mercer style)
import numpy as np

# Prepare counts for 1..4
ng1, ctx1, V1 = counts[1]
ng2, ctx2, V2 = counts[2]
ng3, ctx3, V3 = counts[3]
ng4, ctx4, V4 = counts[4]

# Helper to get MLE conditional probabilities with add-epsilon for stability
def mle_prob(ng_counts, ctx_counts, ngram, eps=0.0, V=len(V4)):
    c = ng_counts.get(ngram, 0)
    if len(ngram) == 1:
        total = sum(ng_counts.values())
        return (c + eps) / (total + eps*V)
    ctx = ngram[:-1]
    denom = ctx_counts.get(ctx, 0) + eps*V
    if denom == 0:
        return 0.0
    return (c + eps) / denom

# Score sentence with interpolation lambdas
def interp_sentence_logprob(sentence, lambdas):
    l1,l2,l3,l4 = lambdas
    tokens = ['<s>']*3 + sentence + ['</s>']
    lp = 0.0
    for i in range(3, len(tokens)):
        w1,w2,w3,w4 = tokens[i-3:i+1]
        p1 = mle_prob(ng1, ctx1, (w4,), eps=1e-8, V=len(V1))
        p2 = mle_prob(ng2, ctx2, (w3,w4), eps=1e-8, V=len(V2))
        p3 = mle_prob(ng3, ctx3, (w2,w3,w4), eps=1e-8, V=len(V3))
        p4 = mle_prob(ng4, ctx4, (w1,w2,w3,w4), eps=1e-8, V=len(V4))
        p = l1*p1 + l2*p2 + l3*p3 + l4*p4
        if p <= 0:
            return float('-inf')
        lp += log(p)
    return lp

# Simple grid search for lambdas that sum to 1
candidates = []
vals = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for l1 in vals:
    for l2 in vals:
        for l3 in vals:
            l4 = 1.0 - (l1+l2+l3)
            if l4 < -1e-9:
                continue
            if l4 < 0: l4 = 0.0
            candidates.append((l1,l2,l3,l4))

# Evaluate on validation set
best_lambdas = None
best_ppl = float('inf')

for lambdas in candidates:
    total_logprob = 0.0
    total_tokens = 0
    for sent in val_set:
        lp = interp_sentence_logprob(sent, lambdas)
        if lp == float('-inf'):
            total_logprob = float('-inf'); break
        total_logprob += lp
        total_tokens += len(sent) + 1
    if total_logprob == float('-inf') or total_tokens == 0:
        ppl = float('inf')
    else:
        avg = total_logprob/total_tokens
        ppl = pow(2.718281828, -avg)
    if ppl < best_ppl:
        best_ppl = ppl
        best_lambdas = lambdas

print(f"Best lambdas (val): {best_lambdas}, perplexity: {best_ppl:.3f}")

# Evaluate best on test
if best_lambdas is not None:
    total_logprob = 0.0
    total_tokens = 0
    for sent in test_set:
        lp = interp_sentence_logprob(sent, best_lambdas)
        if lp == float('-inf'):
            total_logprob = float('-inf'); break
        total_logprob += lp
        total_tokens += len(sent) + 1
    test_ppl = float('inf') if (total_logprob == float('-inf') or total_tokens==0) else pow(2.718281828, -(total_logprob/total_tokens))
    print(f"Test perplexity with best lambdas: {test_ppl:.3f}")

Best lambdas (val): (0.3, 0.2, 0.2, 0.30000000000000004), perplexity: 2217.563
Test perplexity with best lambdas: 1860.701
